In [ ]:
!pip install telebot transformers sentencepiece sacremoses pymystem3 nltk

In [10]:
import telebot
from telebot import types
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import tensorflow as tf
from transformers import pipeline
import pandas as pd
import ast
import json
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem

In [ ]:
token='   '
bot=telebot.TeleBot(token)
@bot.message_handler(commands=['start'])
def start_message(message):
  global recognized_items
  recognized_items = []
  last_button_pressed = None
  bot.send_message(message.chat.id,"Добро пожаловать! Я - твой помощник на кухне. С моей помощью ты сможешь не только распознать продукты по фотографии, но и найти уникальные рецепты, которые подойдут именно для этих продуктов. Давай начнем! Просто отправь фото продукта")

# Создайте глобальный список для хранения распознанных фруктов и овощей
recognized_items = []

#Загружает изображение с заданным размером, преобразует изображение в массив numpy, 
#добавляет дополнительное измерение к массиву, нормализует значения пикселей 
def get_img_array(img_path, size):
   img = tf.keras.preprocessing.image.load_img(img_path, target_size=size)
   array = tf.keras.preprocessing.image.img_to_array(img)
   array = np.expand_dims(array, axis=0)
   return array/255

#Загружает предварительно обученную модель из файла
model = load_model('my_classification_model.h5')

#Загружает изображение и преобразует его в массив с размерами (224, 224) с помощью функции get_img_array.
def recognize_image(image_path):
    img = get_img_array(image_path, (224, 224))
    #Определяет метки классов для различных видов продуктов.
    class_labels = ['apple', 'banana', 'beetroot', 'bell pepper', 'cabbage',
                   'capsicum', 'carrot', 'cauliflower', 'chilli pepper', 'corn',
                   'cucumber', 'egg', 'plant garlic', 'ginger', 'grapes',
                   'jalepeno', 'kiwi', 'lemon', 'lettuce', 'mango',
                   'onion', 'orange', 'paprika', 'pear', 'peas',
                   'pineapple', 'pomegranate', 'potato', 'raddish', 'soy beans',
                   'spinach', 'sweetcor', 'sweetpotato', 'tomato', 'turnip',
                   'watermelon']
    #Использует модель для предсказания класса объекта на изображении.
    preds = model.predict(img)
    pred_class = np.argmax(preds[0])
    #Использует модель перевода для перевода названия класса с английского на русский.
    translator = pipeline("translation_en_to_ru", model="Helsinki-NLP/opus-mt-en-ru")

    # Переведите название на русский язык
    results = translator([class_labels[pred_class]])
    pred_class_russian = results[0]['translation_text']

    # Добавьте распознанный элемент в список
    recognized_items.append(pred_class_russian)

    return f"Продукт на фото: {pred_class_russian}"


@bot.message_handler(content_types=['photo'])
def handle_docs_photo(message):
    #Функция get_file вызывается с идентификатором файла последней фотографии в сообщении. Это возвращает информацию о файле.
    file_info = bot.get_file(message.photo[len(message.photo)-1].file_id)
    #Функция download_file вызывается с путем к файлу, чтобы загрузить файл.
    downloaded_file = bot.download_file(file_info.file_path)

    #Файл сохраняется на диск
    with open('image.jpg', 'wb') as new_file:
      new_file.write(downloaded_file)

    # Предсказание класса изображения
    prediction = recognize_image('image.jpg')

    # Отправка предсказания пользователю
    bot.reply_to(message, prediction)

    # Создание кнопок для первого этапа
    markup = types.InlineKeyboardMarkup()
    item1 = types.InlineKeyboardButton("Загрузить еще фото", callback_data='load')
    item2 = types.InlineKeyboardButton("Нет", callback_data='no')
    markup.add(item1, item2)

    bot.send_message(message.chat.id, 'Вы хотите загрузить еще фото?', reply_markup=markup)

last_button_pressed = None


@bot.callback_query_handler(func=lambda call: True)
def callback_inline(call):
    global recognized_items, last_button_pressed
    try:
        if call.message:
            if call.data == 'load':
                # Если пользователь хочет загрузить еще фото, отправляем ему сообщение и продолжаем ждать новых фото
                bot.send_message(call.message.chat.id, 'Пожалуйста, загрузи фото продукта.')
            elif call.data == 'no':
                # Если пользователь не хочет загрузить еще фото, выводим список и переходим к следующему этапу
                bot.send_message(call.message.chat.id, 'Список распознанных ингредиентов')
                bot.send_message(call.message.chat.id, ', '.join(recognized_items))

                markup = types.InlineKeyboardMarkup()
                item1 = types.InlineKeyboardButton("Добавь еще ингредиенты в этот список", callback_data='add')
                item2 = types.InlineKeyboardButton("Распознанный список ингредиентов верен", callback_data='right')
                item3 = types.InlineKeyboardButton("Напиши свой список ингредиентов", callback_data='write')
                markup.add(item1)
                markup.add(item2)
                markup.add(item3)

                bot.send_message(call.message.chat.id, 'Далее можно откорректировать список либо создать свой список ингредиентов', reply_markup=markup)

            elif call.data == 'add':
                # Если пользователь хочет добавить ингредиенты в список, отправляем ему сообщение и ждем новых ингредиентов
                bot.send_message(call.message.chat.id, 'Пожалуйста, напиши ингредиенты, которые ты хочешь добавить в список.')
                last_button_pressed = 'add'

            elif call.data == 'right':
                # Отправьте список распознанных ингредиентов пользователю
                bot.send_message(call.message.chat.id, 'Список распознанных ингредиентов:')
                bot.send_message(call.message.chat.id, ', '.join(recognized_items))

                # Вызов функции, которая возвращает топ-5 блюд с наибольшим количеством совпадающих ингредиентов
                file_path = './povarenok.csv'
                top_5_dishes = top_5_dishes_with_matching_ingredients(file_path, recognized_items)

                # Создайте кнопки для каждого блюда и отправьте их пользователю
                markup = types.InlineKeyboardMarkup()
                for index, row in top_5_dishes.iterrows():
                    item = types.InlineKeyboardButton(row['name'], url=row['url'])
                    markup.add(item)

                bot.send_message(call.message.chat.id, 'Рекомендуемые блюда:', reply_markup=markup)
                bot.send_message(call.message.chat.id, 'Хочешь начать заново?   Нажми  /start .')

            elif call.data == 'write':
                print("Button 'write' was pressed")
                # Если пользователь хочет написать свой список, обнуляем список recognized_items и ждем новый список
                recognized_items = []
                bot.send_message(call.message.chat.id, 'Пожалуйста, напиши свой список ингредиентов.')
                last_button_pressed = 'write'

    except Exception as e:
        print("An error occurred:", e)
        #print(e)


@bot.message_handler(func=lambda message: True)
def handle_message(message):
    global recognized_items, last_button_pressed
    if last_button_pressed == 'add':
        # Если последней нажатой кнопкой была "add", добавляем сообщение пользователя в список
        recognized_items.append(message.text)
        bot.send_message(message.chat.id, ', '.join(recognized_items))

        file_path = './povarenok.csv'
        top_5_dishes = top_5_dishes_with_matching_ingredients(file_path, recognized_items)

        # Создайте кнопки для каждого блюда
        markup = types.InlineKeyboardMarkup()
        for index, row in top_5_dishes.iterrows():
            item = types.InlineKeyboardButton(row['name'], url=row['url'])
            markup.add(item)

        bot.send_message(message.chat.id, 'Рекомендуемые блюда:', reply_markup=markup)
        bot.send_message(message.chat.id, 'Хочешь начать заново?   Нажми  /start .')

    elif last_button_pressed == 'right':
        # Если последней нажатой кнопкой была "right"
        bot.send_message(message.chat.id, 'Список распознанных ингредиентов')
        recognized_items = message.text.split(',')
        bot.send_message(message.chat.id, ', '.join(recognized_items))

        file_path = './povarenok.csv'
        top_5_dishes = top_5_dishes_with_matching_ingredients(file_path, recognized_items)

        # Создайте кнопки для каждого блюда
        markup = types.InlineKeyboardMarkup()
        for index, row in top_5_dishes.iterrows():
            item = types.InlineKeyboardButton(row['name'], url=row['url'])
            markup.add(item)

        bot.send_message(message.chat.id, 'Рекомендуемые блюда:', reply_markup=markup)
        bot.send_message(message.chat.id, 'Хочешь начать заново?   Нажми  /start .')

    elif last_button_pressed == 'write':
        # Если последней нажатой кнопкой была "write", заменяем список сообщением пользователя
        recognized_items = [item.strip() for sublist in message.text.split(',') for item in sublist.split()]
        bot.send_message(message.chat.id, ', '.join(recognized_items))

        file_path = './povarenok.csv'
        top_5_dishes = top_5_dishes_with_matching_ingredients(file_path, recognized_items)

        # Создайте кнопки для каждого блюда
        markup = types.InlineKeyboardMarkup()
        for index, row in top_5_dishes.iterrows():
            item = types.InlineKeyboardButton(row['name'], url=row['url'])
            markup.add(item)

        bot.send_message(message.chat.id, 'Рекомендуемые блюда:', reply_markup=markup)
        bot.send_message(message.chat.id, 'Хочешь начать заново?   Нажми  /start .')

        last_button_pressed = None

from pymystem3 import Mystem
def top_5_dishes_with_matching_ingredients(file_path, recognized_items):
    # Загрузите данные из файла Excel
    df = pd.read_csv(file_path)
    # Создайте объект стеммера для русского языка
    mystem = Mystem()
    # Замена пропущенных значений на пустой словарь
    df['ingredients'] = df['ingredients'].fillna('{}')
    # Преобразование строк в словари
    df['ingredients'] = df['ingredients'].apply(ast.literal_eval)
    # Создание нового столбца с ключами словаря
    df['keys'] = df['ingredients'].apply(lambda x: ', '.join(list(x.keys())))
    df = df.drop('ingredients', axis=1, inplace=False)
    # Преобразуйте все строки в столбце 'keys' к нижнему регистру
    df['keys'] = df['keys'].str.lower()
    recognized_items = [mystem.lemmatize(ingredient.lower())[0] for ingredient in recognized_items]
    # Создайте новый столбец, который будет содержать количество совпадающих продуктов в каждой строке
    df['matching_count'] = df['keys'].apply(lambda x: sum(1 for ingredient in recognized_items if ingredient in x))
    # Отсортируйте таблицу по столбцу 'matching_count' в убывающем порядке
    df = df.sort_values(by='matching_count', ascending=False)
    # Выведите первые 5 блюд с наибольшим количеством совпадающих продуктов
    top_5_dishes = df.head(5)
    return top_5_dishes


bot.infinity_polling()